In [6]:
import pandas as pd

from google.cloud import bigquery
#from colorama import Fore, Style
#from pathlib import Path

In [ ]:
#df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/data.csv')
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/PubMed.csv')

In [ ]:
df.head()

In [7]:
# Variable
GCP_PROJECT_SEBT84='useful-proposal-392710'
GCP_REGION='europe-west1'

# BigQuery
BQ_REGION='EU'
BQ_DATASET='enhancing_systemic_reviews_nlp'
TABLE = 'pubmed'

# Cloud Storage
BUCKET_NAME='enhancing_systemic_reviews_nlp'

In [8]:
full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.{TABLE}"

client = bigquery.Client()
query_job = client.query(
        f"""
        SELECT
          *
        FROM {GCP_PROJECT_SEBT84}.{BQ_DATASET}.raw_{TABLE}
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
df = results.to_dataframe()

In [9]:
df

,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,1531328,1531328_0_3,To determine the efficacy of nicotinamide in i...,0,3,OBJECTIVE
1,23819670,23819670_2_3,"Group psychotherapy is an effective , evidence...",2,3,CONCLUSIONS
2,10169635,10169635_2_3,"Excess mortality , measured as mortality ratio...",2,3,CONCLUSIONS
3,17894671,17894671_2_3,The author challenges psychiatric nurse clinic...,2,3,CONCLUSIONS
4,18416075,18416075_0_4,To observe therapeutic effects of the comprehe...,0,4,OBJECTIVE
5,12931378,12931378_0_4,To evaluate the validity and security of Feili...,0,4,OBJECTIVE
6,12024594,12024594_0_4,To explore the effect of intravascular low lev...,0,4,OBJECTIVE
7,15023585,15023585_0_4,One of the few combination approaches to the t...,0,4,BACKGROUND
8,14657456,14657456_0_4,Whether cerebral protection during carotid ang...,0,4,OBJECTIVE
9,21837955,21837955_0_4,To compare the clinical effects of transurethr...,0,4,OBJECTIVE


In [20]:
def load_data_bq(df: pd.DataFrame
                 , replace: bool):
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.concat_{TABLE}"

    client = bigquery.Client()

    write_mode = "WRITE_TRUNCATE" if replace else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(df, full_table_name, job_config=job_config)
    result = job.result()

    print(f"✅ Data saved to bigquery, with shape {df.shape}")

In [ ]:
def preprocess_data(path_to_csv, frac):
    """provide the path to csv and the frac of data you want to use and it will
    return the preprocessed dataset in pd.DataFrame format"""

    assert isinstance(path_to_csv, str), "path_to_csv should be a string"
    df = pd.read_csv(path_to_csv)
    df = df.sort_values(by=['abstract_id', 'line_number'])
    df['abstract_text'] = df['abstract_text'].astype(str)
    concatenated_abstract = df.groupby('abstract_id')['abstract_text'].apply(' '.join).reset_index()
    df = df.merge(concatenated_abstract, on='abstract_id', how='left')
    data = df[['abstract_id', 'abstract_text_y']].drop_duplicates().rename(columns={'abstract_text_y': 'abstract_text'})
    data = data.sample(frac=frac).reset_index(drop=True)

    return data

In [11]:
def preprocess_data(df, frac=0.2):
    """provide the path to csv and the frac of data you want to use and it will
    return the preprocessed dataset in pd.DataFrame format"""

    assert isinstance(df, pd.DataFrame), "path_to_csv should be a string"
    
    df = df.sort_values(by=['abstract_id', 'line_number'])
    df['abstract_text'] = df['abstract_text'].astype(str)
    concatenated_abstract = df.groupby('abstract_id')['abstract_text'].apply(' '.join).reset_index()
    df = df.merge(concatenated_abstract, on='abstract_id', how='left')
    data = df[['abstract_id', 'abstract_text_y']].drop_duplicates().rename(columns={'abstract_text_y': 'abstract_text'})
    #data = data.sample(frac=frac).reset_index(drop=True)

    return data

In [13]:
df_2 = preprocess_data(df)

In [21]:
load_data_bq(df_2,replace=True)

✅ Data saved to bigquery, with shape (10, 2)


In [57]:
def get_data_from_bq(table):
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.concat_{TABLE}"

    client = bigquery.Client(project=GCP_PROJECT_SEBT84)
    query = f"""
        SELECT
          count(*)
        FROM {full_table_name}
        """
    query_job = client.query(query)
    
    results = query_job.result()  # Waits for job to complete.
    df = results.to_dataframe()
    count = int(df.values)
    return count

In [69]:
full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.concat_{TABLE}"
frac = 5
client = bigquery.Client(project=GCP_PROJECT_SEBT84)
query = f"""
        SELECT
          *
        FROM {full_table_name}
        limit {frac}
        """
query_job = client.query(query)
    
results = query_job.result()  # Waits for job to complete.
df = results.to_dataframe()
df

,abstract_id,abstract_text
0,1531328,To determine the efficacy of nicotinamide in i...
1,10169635,"Excess mortality , measured as mortality ratio..."
2,12024594,To explore the effect of intravascular low lev...
3,12931378,To evaluate the validity and security of Feili...
4,14657456,Whether cerebral protection during carotid ang...


In [59]:
size = get_data_from_bq(TABLE)

In [62]:
round(size * 0.333)

3